# Import des modules


In [334]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OrdinalEncoder,MinMaxScaler

#Selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold, StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn.utils.class_weight import compute_sample_weight
import shap

#Modèles
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#Metriques
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, average_precision_score, roc_curve, auc

In [335]:
fc = pd.read_csv('fc_after_feature_engineering.csv')
print(fc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 48 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   a_quitte_l_entreprise                      1470 non-null   int64  
 1   age                                        1470 non-null   int64  
 2   annees_dans_l_entreprise                   1470 non-null   int64  
 3   annees_dans_le_poste_actuel                1470 non-null   int64  
 4   annees_depuis_la_derniere_promotion        1470 non-null   int64  
 5   annees_experience_totale                   1470 non-null   int64  
 6   annes_sous_responsable_actuel              1470 non-null   int64  
 7   augmentation_salaire_precedente            1470 non-null   int64  
 8   departement_Commercial                     1470 non-null   int64  
 9   departement_Consulting                     1470 non-null   int64  
 10  departement_Ressources H

# Déclaration des Dataframes X et y
- Un DataFrame contenant les features => X
- Un Pandas Series contenant la colonne cible => y

In [336]:
#Un homme , Célibataire , Entre 30 et 40 ans , Consultant , Entre 1 et 7 années dans l’entreprise, Un revenu compris entre 2500€ et 6000€, Domaine d’etude Infra & Cloud, Qui a moins de 5 années sous son responsable actuel, Qui a une distance domicile travail entre 3 et 17km

# Test 1
#columns_base = ['genre', 'statut_marital', 'age', 'annees_dans_l_entreprise', 'revenu_mensuel', 'distance_domicile_travail', 'satisfaction_globale']
# Test 2
columns_base = ['genre', 'revenu_mensuel', 'age',  'annees_dans_l_entreprise', 'distance_domicile_travail', 'score_salaire', 'satisfaction_globale']

columns_domaine_etude = [col for col in fc.columns if col.startswith('domaine_etude')]
columns_poste = [col for col in fc.columns if col.startswith('poste')]
columns_statut_marital = [col for col in fc.columns if col.startswith('statut_marital')]
columns_satisfaction = [col for col in fc.columns if col.startswith('satisfaction')]

all_columns = columns_base + columns_statut_marital + columns_domaine_etude + columns_poste + columns_satisfaction

X = fc[all_columns]
y = fc['a_quitte_l_entreprise']

# Fonction de Séparation Train Test
- Des métriques d’évaluation calculées pour chaque modèle, sur le jeu d’apprentissage et le jeu de test.

In [337]:
def train_test_evaluation(model, X_train, X_test, y_train, y_test, model_name, n_shap_samples=5):
    """
    Entraîne un modèle et retourne les métriques train/test avec détection d'overfitting.

    Args:
        model: Le modèle à entraîner
        X_train, X_test: Features d'entraînement et de test
        y_train, y_test: Labels d'entraînement et de test
        model_name: Nom du modèle (string)
        n_shap_samples: Nombre d'individus à analyser avec SHAP (défaut: 5)

    Returns:
        tuple: (results, report_test, cm, curves, df_importances, importances_metadata)
    """

    # Poids
    weights = compute_sample_weight(class_weight='balanced', y=y_train)

    # Création d'un pipeline avec standardisation puis modèle
    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),  # Normalisation des features
        ('model', model)               # Modèle de régression
    ])

    # Entraînement avec les poids passés au modèle via le préfixe 'model__'
    pipeline.fit(X_train, y_train, model__sample_weight=weights)

    # Prédictions sur TRAIN
    y_pred_train = pipeline.predict(X_train)
    report_train = classification_report(y_train, y_pred_train, output_dict=True)

    # Prédictions sur TEST
    y_pred_test = pipeline.predict(X_test)
    report_test = classification_report(y_test, y_pred_test, output_dict=True)

    # Matrice de confusion (test)
    cm = confusion_matrix(y_test, y_pred_test)

    # Calcul des écarts (train - test) pour détecter l'overfitting
    accuracy_gap = report_train['accuracy'] - report_test['accuracy']
    f1_gap = report_train['macro avg']['f1-score'] - report_test['macro avg']['f1-score']

    # Indicateur d'overfitting (seuil à 12% d'écart)
    overfitting_flag = 'OUI' if (accuracy_gap > 0.12 or f1_gap > 0.12) else 'NON'

    # Extraction des métriques principales
    results = {
        'model': model_name,
        'method': 'train_test',
        # Métriques TRAIN
        'train_accuracy': round(report_train['accuracy'], 2),
        'train_f1_macro': round(report_train['macro avg']['f1-score'], 2),
        'train_precision_macro': round(report_train['macro avg']['precision'], 2),
        'train_recall_macro': round(report_train['macro avg']['recall'], 2),
        # Métriques TEST
        'test_accuracy': round(report_test['accuracy'], 2),
        'test_f1_macro': round(report_test['macro avg']['f1-score'], 2),
        'test_precision_macro': round(report_test['macro avg']['precision'], 2),
        'test_recall_macro': round(report_test['macro avg']['recall'], 2),
        # Écarts et overfitting
        'accuracy_gap': round(accuracy_gap, 2),
        'f1_gap': round(f1_gap, 2),
        'overfitting': overfitting_flag,
        'confusion_matrix': str(cm.tolist()),
    }

    # Récupérer le modèle entraîné du pipeline
    trained_model = pipeline.named_steps['model']

    # Transformer X_test avec le scaler avant de l'utiliser avec le modèle
    X_test_scaled = pipeline.named_steps['scaler'].transform(X_test)

    # Vérifier si le modèle supporte predict_proba
    try:
        y_pred_proba = trained_model.predict_proba(X_test_scaled)
    except AttributeError:
        y_pred_proba = None

    # Récupération des données pour les courbes ROC et PR
    curves = {
        'model': model_name,
        'y_true': y_test.copy() if hasattr(y_test, 'copy') else list(y_test),
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred_test
    }

    # Permutation importance sur les données scalées
    pi = permutation_importance(trained_model, X_test_scaled, y_test, n_repeats=10, random_state=82)

    # SHAP et feature importances (uniquement pour modèles tree-based)
    tree_based_models = (
        XGBClassifier
    )

    if isinstance(trained_model, tree_based_models):
        explainer = shap.TreeExplainer(trained_model)
        shap_values = explainer.shap_values(X_test_scaled)

        # Récupérer les noms de features
        feature_names_in = trained_model.feature_names_in_ if hasattr(trained_model, 'feature_names_in_') else X_test.columns.tolist()
        feature_importances = trained_model.feature_importances_
    else:
        shap_values = None
        feature_names_in = X_test.columns.tolist()
        feature_importances = None
        print(f"⚠ Modèle ignoré (non compatible avec TreeExplainer): {type(trained_model).__name__}")

    # Construction du DataFrame d'importances
    df_importances = pd.DataFrame({
        'model': model_name,
        'feature': X_test.columns,
        'importance_mean': pi.importances_mean,
        'importance_std': pi.importances_std
    })

    # Ajouter les feature importances si disponibles
    if feature_importances is not None:
        df_importances['feature_importances'] = feature_importances

    # Stocker les autres informations séparément
    importances_metadata = {
        'model': model_name,
        'X_test': X_test,  # Garder les données non-scalées pour référence
        'X_test_scaled': X_test_scaled,  # Ajouter les données scalées
        'feature_names_in': feature_names_in,
        'shap_values': shap_values
    }

    return results, report_test, cm, curves, df_importances, importances_metadata

# Fonction de Validation Croisée
- cross_validate

In [338]:
def cross_validation_evaluation(model, X, y, model_name, cv=5):
    """
    Effectue une validation croisée et retourne les métriques avec détection d'overfitting.

    Args:
        model: Le modèle à évaluer
        X: Features complètes
        y: Labels complets
        model_name: Nom du modèle (string)
        cv: Nombre de folds (défaut: 5)

    Returns:
        dict: Dictionnaire contenant les résultats moyens avec scores train et test
    """

    # Création du pipeline avec scaling
    pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
    ])

    # Définition des métriques à calculer
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

    # Validation croisée avec return_train_score=True pour détecter overfitting
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=True, error_score='raise')

    # Calcul des écarts moyens (train - test)
    accuracy_gap_cv = cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean()
    f1_gap_cv = cv_results['train_f1_macro'].mean() - cv_results['test_f1_macro'].mean()

    # Indicateur d'overfitting
    overfitting_flag_cv = 'OUI' if (accuracy_gap_cv >= 0.12 or f1_gap_cv >= 0.12) else 'NON'

    # Calcul des moyennes et écarts-types
    results = {
        'model': model_name,
        'method': 'cross_validation',
        # Métriques TRAIN
        'train_accuracy': round(cv_results['train_accuracy'].mean(), 2),
        'train_accuracy_std': round(cv_results['train_accuracy'].std(), 2),
        'train_f1_macro': round(cv_results['train_f1_macro'].mean(), 2),
        'train_f1_macro_std': round(cv_results['train_f1_macro'].std(), 2),
        'train_precision_macro': round(cv_results['train_precision_macro'].mean(), 2),
        'train_recall_macro': round(cv_results['train_recall_macro'].mean(), 2),
        # Métriques TEST
        'test_accuracy': round(cv_results['test_accuracy'].mean(), 2),
        'test_accuracy_std': round(cv_results['test_accuracy'].std(), 2),
        'test_f1_macro': round(cv_results['test_f1_macro'].mean(), 2),
        'test_f1_macro_std': round(cv_results['test_f1_macro'].std(), 2),
        'test_precision_macro': round(cv_results['test_precision_macro'].mean(), 2),
        'test_recall_macro': round(cv_results['test_recall_macro'].mean(), 2),
        # Écarts et overfitting
        'accuracy_gap': round(accuracy_gap_cv, 2),
        'f1_gap': round(f1_gap_cv, 2),
        'overfitting': overfitting_flag_cv,
        'confusion_matrix': 'N/A',
    }

    return results

# Fonction de Comparaison des Modeles

- DummyClassifier : strategy='most_frequent'
- RandomForestClassifier : n_estimators=100,max_depth=10
- XGBClassifier : eval_metric='logloss'
- CatBoostClassifier

In [339]:
def compare_models(X, y, test_size=0.2, random_state=82, cv_folds=5):
    """
    Compare tous les modèles et génère les fichiers CSV de résultats.

    Args:
        X: Features (DataFrame ou array)
        y: Labels (Series ou array)
        test_size: Proportion du jeu de test (défaut: 0.2)
        random_state: Seed pour la reproductibilité (défaut: 42)
        cv_folds: Nombre de folds pour la validation croisée (défaut: 5)

    Returns:
        tuple: (résultats_df, rapports_détaillés)
    """
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Définition des modèles
    models = {
        'DummyClassifier': DummyClassifier(
            strategy='most_frequent',
            random_state=random_state),

        'RandomForestRegressor': RandomForestRegressor(
            n_estimators=100,
            max_depth=5,
            min_samples_split=2,
            random_state=random_state),

        'XGBClassifier': XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1,     # Plus petit = apprentissage plus lent
            subsample=0.8,          # 80% des données par arbre
            reg_lambda=2,         # Régularisation L2
            eval_metric='logloss',
            scale_pos_weight=5,
            random_state=random_state)

        #'CatBoostClassifier': CatBoostClassifier(
        #    iterations=300,
        #    depth=4,
        #    learning_rate=0.05,
        #    l2_leaf_reg=3,        # Régularisation
        #    verbose=False,
        #    random_state=random_state)
    }

    all_results = []
    detailed_reports = []
    all_curves = []
    all_df_importances = []
    all_importances_metadata = []

    print("=" * 70)
    print("COMPARAISON DES MODÈLES DE CLASSIFICATION")
    print("=" * 70)

    for model_name, model in models.items():
        print(f"\n>>> Évaluation de {model_name}...")

        # Train/Test
        print(f"  - Train/Test split...")
        tt_results, report, cm, curves, df_importances,  importances_metadata = train_test_evaluation(model, X_train, X_test, y_train, y_test, model_name)
        all_results.append(tt_results)

        # Sauvegarde du rapport détaillé
        detailed_reports.append({
            'model': model_name,
            'method': 'train_test',
            'report': report,
            'confusion_matrix': cm
        })

        # Sauvegarde des courbes
        all_curves.append(curves)

        # Sauvegarde des courbes
        all_df_importances.append(df_importances)

        # Sauvegarde des courbes
        all_importances_metadata.append(importances_metadata)

        # Cross-validation
        print(f"  - Validation croisée ({cv_folds} folds)...")
        cv_results = cross_validation_evaluation(model, X, y, model_name, cv=cv_folds)
        all_results.append(cv_results)

        print(f"  ✓ {model_name} terminé")

    # Conversion en DataFrame
    results_df = pd.DataFrame(all_results)

    # Réorganisation des colonnes pour la lisibilité
    cols_order = ['model', 'method',
                  'train_accuracy', 'test_accuracy', 'accuracy_gap',
                  'train_f1_macro', 'test_f1_macro', 'f1_gap',
                  'overfitting',
                  'train_precision_macro', 'test_precision_macro',
                  'train_recall_macro', 'test_recall_macro']

    # Ajout des colonnes std si elles existent
    std_cols = [col for col in results_df.columns if '_std' in col]
    cols_order.extend(std_cols)
    cols_order.append('confusion_matrix')

    # Colonnes présentes dans le DataFrame
    cols_order = [col for col in cols_order if col in results_df.columns]
    results_df = results_df[cols_order]

    return results_df, detailed_reports, all_curves, all_df_importances, all_importances_metadata

# Fonction de Sauvegarde des Résultats
- classification_results_by_class.csv => Rapport de classification par classes
- classification_results_confusion_matrices.csv => Matrices de confusion
- classification_results_summary.csv => Tous les scores
- classification_results_overfitting_analysis.csv => Analyse spécifique de l'overfitting

In [340]:
def save_results(results_df, detailed_reports, output_prefix, all_curves=None, all_df_importances=None, all_importances_metadata=None):
    """
    Sauvegarde les résultats dans des fichiers CSV.

    Args:
        results_df: DataFrame avec tous les résultats
        detailed_reports: Liste des rapports détaillés
        output_prefix: Préfixe pour les fichiers de sortie
               models_data: Liste de dict avec {
            'model': nom du modèle,
            'y_true': vraies étiquettes,
            'y_pred_proba': probabilités prédites (si disponible),
            'y_pred': prédictions
        }
    """
    # 1. Fichier principal avec toutes les métriques
    results_df.to_csv(f'exports/{output_prefix}_summary.csv', index=False)
    print(f"\n✓ Résumé sauvegardé: {output_prefix}_summary.csv")

    # 2. Fichier avec les rapports détaillés de classification
    detailed_data = []
    for item in detailed_reports:
        model = item['model']
        report = item['report']

        # Extraction des métriques par classe
        for class_label, metrics in report.items():
            if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
                detailed_data.append({
                    'model': model,
                    'class': class_label,
                    'precision': round(metrics['precision'], 2),
                    'recall': round(metrics['recall'], 2),
                    'f1-score': round(metrics['f1-score'], 2),
                    'support': metrics['support']
                })

    detailed_df = pd.DataFrame(detailed_data)
    detailed_df.to_csv(f'exports/{output_prefix}_by_class.csv', index=False)
    print(f"✓ Résultats par classe sauvegardés: {output_prefix}_by_class.csv")


    # 4. Fichier spécifique pour l'analyse d'overfitting
    overfitting_data = results_df[['model', 'method', 'train_accuracy', 'test_accuracy',
                                     'accuracy_gap', 'train_f1_macro', 'test_f1_macro',
                                     'f1_gap', 'overfitting']].copy()
    overfitting_data.to_csv(f'exports/{output_prefix}_overfitting_analysis.csv', index=False)
    print(f"✓ Analyse d'overfitting sauvegardée: {output_prefix}_overfitting_analysis.csv")

    # 5. Génération des graphiques ROC et Précision-Rappel
    if all_curves is not None:
        print("\n📈 Génération des graphiques ROC et Précision-Rappel...")
        _plot_curves(all_curves, output_prefix)

    # 6. Génération des graphiques Importances
    if all_df_importances is not None:
        print("\n📈 Génération des graphiques Importances...")
        _plot_importances(all_df_importances, all_importances_metadata, output_prefix)

    print("\n" + "=" * 70)
    print("TOUS LES RÉSULTATS ONT ÉTÉ SAUVEGARDÉS")
    print("=" * 70)

    # Affichage d'un résumé de l'overfitting
    print("\n📊 RÉSUMÉ DE L'OVERFITTING:")
    print("-" * 70)
    for _, row in overfitting_data.iterrows():
        status = "⚠️  OVERFITTING DÉTECTÉ" if row['overfitting'] == 'OUI' else "✓  Pas d'overfitting"
        print(f"{row['model']:25s} ({row['method']:17s}): {status}")
        print(f"  → Écart accuracy: {row['accuracy_gap']:+.4f} | Écart F1: {row['f1_gap']:+.4f}")
    print("-" * 70)


# Fonctions Génération Graphiques ROC et PR
- Courbes ROC pour classification binaire.
- Courbes Précision-Rappel pour classification binaire.

In [341]:
def _plot_curves(all_curves, output_prefix):
    """
    Génère et sauvegarde les courbes pour tous les modèles.
    """

    _plot_binary_roc_curves(all_curves, output_prefix)
    _plot_binary_pr_curves(all_curves, output_prefix)

def _plot_binary_roc_curves(all_curves, output_prefix):
    """
    Courbes ROC pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in all_curves:
        model_name = data['model']
        y_true = data['y_true']

        # Vérifier si les probabilités sont disponibles
        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            # Si format (n_samples, n_classes), prendre la colonne de la classe positive
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        # Calculer la courbe ROC
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

    # Ligne diagonale (classificateur aléatoire)
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Aléatoire (AUC = 0.500)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de Faux Positifs', fontsize=12)
    plt.ylabel('Taux de Vrais Positifs', fontsize=12)
    plt.title('Courbes ROC - Comparaison des Modèles', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_roc_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes ROC sauvegardées: {output_prefix}_roc_curves.png")

def _plot_binary_pr_curves(all_curves, output_prefix):
    """
    Courbes Précision-Rappel pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in all_curves:
        model_name = data['model']
        y_true = data['y_true']

        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        avg_precision = average_precision_score(y_true, y_score)

        plt.plot(recall, precision, lw=2,
                 label=f'{model_name} (AP = {avg_precision:.3f})')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Rappel', fontsize=12)
    plt.ylabel('Précision', fontsize=12)
    plt.title('Courbes Précision-Rappel - Comparaison des Modèles',
              fontsize=14, fontweight='bold')
    plt.legend(loc="lower left", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_precision_recall_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes Précision-Rappel sauvegardées: {output_prefix}_precision_recall_curves.png")

# Fonctions Génération Graphiques Importance Features
-
-

# Affichage des Résultats

In [342]:
def _plot_importances(all_df_importances, all_importances_metadata, output_prefix):
    """
    Génère et sauvegarde les courbes pour tous les modèles.

    Args:
        all_df_importances: Liste de DataFrames contenant les importances de permutation
        all_importances_metadata: Liste de dictionnaires contenant les métadonnées (shap, feature_importances, etc.)
        output_prefix: Préfixe pour les noms de fichiers
    """
    _plot_permutation_importance(all_df_importances, output_prefix)
    _plot_native_feature_importance(all_df_importances, all_importances_metadata, output_prefix)
    _plot_shap(all_importances_metadata, output_prefix)
    _plot_shap_waterfall_combined(all_importances_metadata, output_prefix)

def _plot_permutation_importance(all_df_importances, output_prefix):
    """
    Plot de la permutation importance pour tous les modèles.
    """
    plt.figure(figsize=(14, 8))

    # Définir une palette de couleurs pour différencier les modèles
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#96CEB4', '#FFEAA7']

    all_df_importances = [df for df in all_df_importances
                      if not df['importance_mean'].isna().all() and (df['importance_mean'] != 0).any()]

    # Récupérer les features du premier DataFrame et trier alphabétiquement
    first_df = all_df_importances[0].copy()
    features = sorted(first_df['feature'].tolist())

    # Position des barres pour chaque modèle
    bar_height = 0.8 / len(all_df_importances)  # Ajuster la hauteur selon le nombre de modèles
    y_positions = range(len(features))

    for idx, df in enumerate(all_df_importances):
        # Trier le DataFrame selon l'ordre des features
        df_sorted = df.set_index('feature').loc[features].reset_index()

        # Récupérer le nom du modèle si disponible
        model_name = df.get('model', [f'Modèle {idx+1}'])[0] if 'model' in df.columns else f'Modèle {idx+1}'

        # Décaler chaque modèle verticalement
        y_offset = [y + idx * bar_height for y in y_positions]

        plt.barh(y_offset, df_sorted['importance_mean'],
                 height=bar_height,
                 label=model_name,
                 color=colors[idx % len(colors)],
                 alpha=0.8)

    # Ajuster les positions des labels sur l'axe Y
    plt.yticks([y + bar_height * (len(all_df_importances) - 1) / 2 for y in y_positions], features)
    plt.xlabel('Importance Mean', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title(f'Permutation Importance - Comparaison des Modèles ({output_prefix})',
              fontsize=14, fontweight='bold')
    plt.legend(loc="best", fontsize=10)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_permutation_importance_comparison.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Graphique de comparaison sauvegardé: {filename}")

def _plot_native_feature_importance(all_df_importances, all_importances_metadata, output_prefix):
    """
    Plot de la feature importance native pour les modèles tree-based,
    avec normalisation de chaque modèle pour les mettre sur la même échelle.
    """

    # Filtrer uniquement les modèles qui ont des feature_importances
    valid_data = [
        (df.copy(), meta) for df, meta in zip(all_df_importances, all_importances_metadata)
        if 'feature_importances' in df.columns and df['feature_importances'].notna().any()
    ]

    if not valid_data:
        print(f"⚠ Aucun modèle avec feature_importances natives trouvé. Graphique ignoré.")
        return

    plt.figure(figsize=(14, 8))

    # Palette de couleurs pour différencier les modèles
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#96CEB4', '#FFEAA7']

    # Normalisation des importances pour chaque modèle (somme = 1)
    for df, _ in valid_data:
        total = df['feature_importances'].sum()
        if total != 0:
            df['feature_importances'] = df['feature_importances'] / total

    # Récupérer la liste ordonnée des features du premier modèle
    first_df = valid_data[0][0].sort_values(by='feature_importances', ascending=True)
    features = first_df['feature'].tolist()

    # Position verticale des barres
    bar_height = 0.8 / len(valid_data)
    y_positions = range(len(features))

    for idx, (df, meta) in enumerate(valid_data):
        # Trier selon l’ordre des features du premier modèle
        df_sorted = df.set_index('feature').loc[features].reset_index()

        # Nom du modèle
        model_name = df.get('model', [f'Modèle {idx + 1}'])[0] if 'model' in df.columns else f'Modèle {idx + 1}'

        # Décalage vertical pour éviter le chevauchement
        y_offset = [y + idx * bar_height for y in y_positions]

        plt.barh(
            y_offset,
            df_sorted['feature_importances'],
            height=bar_height,
            label=model_name,
            color=colors[idx % len(colors)],
            alpha=0.8
        )

    # Mise en forme
    plt.yticks([y + bar_height * (len(valid_data) - 1) / 2 for y in y_positions], features)
    plt.xlabel('Feature Importance (normalisée)', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title(f'Native Feature Importance Normalisée - Comparaison des Modèles ({output_prefix})',
              fontsize=14, fontweight='bold')
    plt.legend(loc="best", fontsize=10)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_native_feature_importance_comparison_normalized.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Graphique normalisé sauvegardé : {filename}")

def _plot_shap(all_importances_metadata, output_prefix):
    """
    Plot des valeurs SHAP pour les modèles tree-based.

    Args:
        all_importances_metadata: Liste de dictionnaires contenant X_test et shap_values
        output_prefix: Préfixe pour les noms de fichiers
    """
    for idx, meta in enumerate(all_importances_metadata):
        if meta['shap_values'] is None:
            continue

        model_name = meta.get('model', f'Modèle {idx + 1}')
        X_test = meta['X_test']
        shap_values = meta['shap_values']

        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_test, show=False)
        plt.title(f'SHAP Summary Plot - {model_name}', fontsize=14, fontweight='bold')
        plt.tight_layout()

        filename = f'images/{output_prefix}_shap_summary_{model_name}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ SHAP summary plot sauvegardé: {filename}")

def _plot_shap_waterfall_combined(all_importances_metadata, output_prefix, n_samples=5):
    """
    Plot combiné des waterfall SHAP pour plusieurs individus sur une seule figure.
    """
    for idx, meta in enumerate(all_importances_metadata):
        shap_plots = meta.get('shap_plots', [])

        if not shap_plots:
            continue

        model_name = meta.get('model', f'Modèle {idx + 1}')
        n_plots = min(n_samples, len(shap_plots))

        # Créer une figure avec subplots
        fig, axes = plt.subplots(n_plots, 1, figsize=(12, 5 * n_plots))
        if n_plots == 1:
            axes = [axes]

        for i in range(n_plots):
            plot_data = shap_plots[i]
            plt.sca(axes[i])

            shap.plots.waterfall(plot_data['explanation'], show=False)

            true_label = plot_data['true_label']
            pred_label = plot_data['predicted_label']
            individual_idx = plot_data['index']
            match_status = "✓" if true_label == pred_label else "✗"

            axes[i].set_title(
                f'Individu {individual_idx} - Vraie: {true_label} | Prédite: {pred_label} {match_status}',
                fontsize=10, fontweight='bold'
            )

        fig.suptitle(f'SHAP Waterfall Plots - {model_name}', fontsize=14, fontweight='bold', y=1.001)
        plt.tight_layout()

        filename = f'images/{output_prefix}_{model_name}_waterfall_combined.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ SHAP waterfall combiné sauvegardé: {filename}")

In [343]:
# Comparaison des modèles
results_df, detailed_reports, all_curves, all_df_importances, all_importances_metadata = compare_models(X, y)

# Sauvegarde des résultats
save_results(
    results_df,
    detailed_reports,
    output_prefix='classification_results',
    all_curves=all_curves,
    all_importances_metadata=all_importances_metadata,
    all_df_importances=all_df_importances
)

COMPARAISON DES MODÈLES DE CLASSIFICATION

>>> Évaluation de DummyClassifier...
  - Train/Test split...
⚠ Modèle ignoré (non compatible avec TreeExplainer): DummyClassifier
  - Validation croisée (5 folds)...
  ✓ DummyClassifier terminé

>>> Évaluation de RandomForestRegressor...
  - Train/Test split...


/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

ValueError: Classification metrics can't handle a mix of binary and continuous targets

# Modele NON LINEAIRE
- RandomForest, XGBoost ou CatBoost
- Métriques d’évaluation en classification : matrice de confusion, rappel et précision.
- Scores (présence d’overfit ou non, capacité d’éviter les faux positifs ou faux négatifs)

# Amélioration de la classification
- demandez-vous si éviter des faux positifs est plus important qu’éviter des faux négatifs.

# Feature Importance

### Feature Importance globale : approche comparative
- Permutation Importance (sklearn) -> PERMUTATION
- Feature Importance native (si modèle à base d’arbre) -> NATIVE
- SHAP Global Importance (Beeswarm Plot) -> SHAP

### Shapley Values
- SHAP Global Importance (Beeswarm Plot) -> SHAP

### Feature Importance local
- Feature Importance locale (Waterfall Plot)

# OPTIMISATION DES HYPER-PARAMETRES

In [232]:
def get_param_grids():
    """
    Définit les grilles de paramètres pour chaque modèle.

    Returns:
        dict: Dictionnaire avec les grilles de paramètres pour chaque modèle
    """
    param_grids = {
        'DummyClassifier': {
            'strategy': ['most_frequent', 'stratified', 'uniform']
        },

        'RandomForestClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3,4,5,6,7, None],
            'min_samples_split': [2, 5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'max_features': ['sqrt', 'log2']
        },

        'XGBClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'reg_alpha': [0, 0.1, 1],
            'reg_lambda': [1, 2, 5]
        },

        'CatBoostClassifier': {
            'iterations': [100, 200, 300],
            'depth': [4, 6, 8],
            'learning_rate': [0.01, 0.05, 0.1],
            'l2_leaf_reg': [1, 3, 5],
            'subsample': [0.8, 1.0]
        }
    }

    return param_grids

def get_small_param_grids():
    """
    Grilles réduites pour des tests rapides.

    Returns:
        dict: Grilles de paramètres réduites
    """
    param_grids = {
        'DummyClassifier': {
            'strategy': ['most_frequent', 'stratified']
        },

        'RandomForestClassifier': {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None],
            'min_samples_split': [2, 10],
            'max_features': ['sqrt']
        },

        'XGBClassifier': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0],
            'reg_lambda': [1, 2]
        },

        'CatBoostClassifier': {
            'iterations': [50, 100],
            'depth': [4, 6],
            'learning_rate': [0.01, 0.1],
            'l2_leaf_reg': [1, 3]
        }
    }

    return param_grids

def perform_grid_search(model, param_grid, X_train, y_train, model_name,
                       cv=5, scoring='f1_macro', n_jobs=-1):
    """
    Effectue un GridSearchCV pour un modèle donné.

    Args:
        model: Le modèle à optimiser
        param_grid: Grille de paramètres
        X_train, y_train: Données d'entraînement
        model_name: Nom du modèle
        cv: Nombre de folds pour la validation croisée
        scoring: Métrique d'optimisation
        n_jobs: Nombre de processus parallèles

    Returns:
        tuple: (GridSearchCV object, résultats dict)
    """
    print(f"\n{'='*70}")
    print(f"🔍 Grid Search pour {model_name}")
    print(f"{'='*70}")
    print(f"Nombre de combinaisons à tester: {np.prod([len(v) for v in param_grid.values()])}")
    print(f"Métrique d'optimisation: {scoring}")
    print(f"Validation croisée: {cv} folds")

    start_time = time.time()

    # Configuration de la validation croisée stratifiée
    cv_strategy = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv_strategy,
        scoring=scoring,
        n_jobs=n_jobs,
        verbose=0,
        return_train_score=True
    )

    # Entraînement
    grid_search.fit(X_train, y_train)

    elapsed_time = time.time() - start_time

    # Résultats
    results = {
        'model': model_name,
        'best_score': round(grid_search.best_score_, 2),
        'best_params': str(grid_search.best_params_),
        'n_combinations': len(grid_search.cv_results_['params']),
        'time_seconds': round(elapsed_time, 0)
    }

    print(f"✓ Terminé en {elapsed_time:.2f} secondes")
    print(f"📊 Meilleur score ({scoring}): {grid_search.best_score_:.4f}")
    print(f"🏆 Meilleurs paramètres:")
    for param, value in grid_search.best_params_.items():
        print(f"   - {param}: {value}")

    return grid_search, results

def evaluate_best_model(grid_search, X_train, X_test, y_train, y_test, model_name):
    """
    Évalue le meilleur modèle trouvé par GridSearch sur le jeu de test.

    Args:
        grid_search: Objet GridSearchCV entraîné
        X_train, X_test: Features
        y_train, y_test: Labels
        model_name: Nom du modèle

    Returns:
        dict: Résultats d'évaluation
    """
    best_model = grid_search.best_estimator_

    # Prédictions
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)

    # Rapports
    report_train = classification_report(y_train, y_pred_train, output_dict=True)
    report_test = classification_report(y_test, y_pred_test, output_dict=True)

    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred_test)

    # Calcul des écarts (overfitting)
    accuracy_gap = report_train['accuracy'] - report_test['accuracy']
    f1_gap = report_train['macro avg']['f1-score'] - report_test['macro avg']['f1-score']
    overfitting = 'OUI' if (accuracy_gap > 0.12 or f1_gap > 0.12) else 'NON'

    results = {
        'model': model_name,
        'train_accuracy': round(report_train['accuracy'], 2),
        'test_accuracy': round(report_test['accuracy'], 2),
        'train_f1_macro': round(report_train['macro avg']['f1-score'], 2),
        'test_f1_macro': round(report_test['macro avg']['f1-score'], 2),
        'train_precision': round(report_train['macro avg']['precision'], 2),
        'test_precision': round(report_test['macro avg']['precision'], 2),
        'train_recall': round(report_train['macro avg']['recall'], 2),
        'test_recall': round(report_test['macro avg']['recall'], 2),
        'accuracy_gap': round(accuracy_gap, 2),
        'f1_gap': round(f1_gap, 2),
        'overfitting': overfitting,
        'confusion_matrix': str(cm.tolist())
    }

    print(f"\n📈 Évaluation sur le jeu de test:")
    print(f"   Train accuracy: {report_train['accuracy']:.4f}")
    print(f"   Test accuracy:  {report_test['accuracy']:.4f}")
    print(f"   Écart accuracy: {accuracy_gap:+.4f}")
    print(f"   Overfitting:    {overfitting}")

    return results, report_test, cm

def compare_models_gridsearch(X, y, param_grids='full', test_size=0.15,
                              cv=5, scoring='f1_macro', random_state=82):
    """
    Compare tous les modèles avec GridSearchCV.

    Args:
        X: Features
        y: Labels
        param_grids: 'full', 'small', ou dict personnalisé
        test_size: Proportion du jeu de test
        cv: Nombre de folds
        scoring: Métrique d'optimisation
        random_state: Seed

    Returns:
        tuple: (résultats_grid, résultats_eval, grid_objects)
    """
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"\n{'='*70}")
    print(f"GRIDSEARCH - COMPARAISON DES MODÈLES")
    print(f"{'='*70}")
    print(f"Taille du dataset: {len(X)} échantillons")
    print(f"Train: {len(X_train)} | Test: {len(X_test)}")
    print(f"Distribution des classes: {dict(pd.Series(y).value_counts())}")

    # Sélection de la grille
    if param_grids == 'full':
        grids = get_param_grids()
    elif param_grids == 'small':
        grids = get_small_param_grids()
    else:
        grids = param_grids

    # Modèles de base
    base_models = {
        'DummyClassifier': DummyClassifier(random_state=random_state),
        'RandomForestClassifier': RandomForestClassifier(random_state=random_state),
        'XGBClassifier': XGBClassifier(random_state=random_state, eval_metric='logloss'),
        'CatBoostClassifier': CatBoostClassifier(random_state=random_state, verbose=False)
    }

    grid_results = []
    eval_results = []
    grid_objects = {}
    detailed_reports = []

    # GridSearch pour chaque modèle
    for model_name, base_model in base_models.items():
        param_grid = grids[model_name]

        # GridSearch
        grid_search, grid_res = perform_grid_search(
            base_model, param_grid, X_train, y_train,
            model_name, cv=cv, scoring=scoring
        )
        grid_results.append(grid_res)
        grid_objects[model_name] = grid_search

        # Évaluation du meilleur modèle
        eval_res, report, cm = evaluate_best_model(
            grid_search, X_train, X_test, y_train, y_test, model_name
        )
        eval_results.append(eval_res)

        detailed_reports.append({
            'model': model_name,
            'report': report,
            'confusion_matrix': cm
        })

    # Conversion en DataFrames
    grid_df = pd.DataFrame(grid_results)
    eval_df = pd.DataFrame(eval_results)

    return grid_df, eval_df, grid_objects, detailed_reports

def save_gridsearch_results(grid_df, eval_df, grid_objects, detailed_reports,
                            output_prefix='gridsearch_results'):
    """
    Sauvegarde tous les résultats du GridSearch.

    Args:
        grid_df: DataFrame avec résultats du GridSearch
        eval_df: DataFrame avec évaluation finale
        grid_objects: Dict des objets GridSearchCV
        detailed_reports: Rapports détaillés
        output_prefix: Préfixe des fichiers
    """
    # 1. Résumé du GridSearch
    grid_df.to_csv(f'exports/{output_prefix}_grid_summary.csv', index=False)
    print(f"\n✓ Résumé GridSearch: {output_prefix}_grid_summary.csv")

    # 2. Évaluation finale
    eval_df.to_csv(f'exports/{output_prefix}_evaluation.csv', index=False)
    print(f"✓ Évaluation finale: {output_prefix}_evaluation.csv")

    # 3. Détails complets du GridSearch (tous les résultats)
    all_cv_results = []
    for model_name, grid_search in grid_objects.items():
        cv_res = pd.DataFrame(grid_search.cv_results_)
        cv_res.insert(0, 'model', model_name)
        all_cv_results.append(cv_res)

    full_cv_df = pd.concat(all_cv_results, ignore_index=True)
    full_cv_df.to_csv(f'exports/{output_prefix}_full_cv_results.csv', index=False)
    print(f"✓ Résultats CV complets: {output_prefix}_full_cv_results.csv")

    # 4. Meilleurs paramètres
    best_params_data = []
    for model_name, grid_search in grid_objects.items():
        for param, value in grid_search.best_params_.items():
            best_params_data.append({
                'model': model_name,
                'parameter': param,
                'value': str(value)
            })

    best_params_df = pd.DataFrame(best_params_data)
    best_params_df.to_csv(f'exports/{output_prefix}_best_params.csv', index=False)
    print(f"✓ Meilleurs paramètres: {output_prefix}_best_params.csv")

    # 5. Matrices de confusion
    #cm_data = []
    #for item in detailed_reports:
    #    cm_data.append({
    #        'model': item['model'],
    #        'confusion_matrix': str(item['confusion_matrix'].tolist())
    #    })
    #cm_df = pd.DataFrame(cm_data)
    #cm_df.to_csv(f'exports/{output_prefix}_confusion_matrices.csv', index=False)
    #print(f"✓ Matrices de confusion: {output_prefix}_confusion_matrices.csv")

    # 6. Analyse d'overfitting
    overfitting_cols = ['model', 'train_accuracy', 'test_accuracy', 'accuracy_gap',
                       'train_f1_macro', 'test_f1_macro', 'f1_gap', 'overfitting']
    overfitting_df = eval_df[overfitting_cols]
    overfitting_df.to_csv(f'exports/{output_prefix}_overfitting.csv', index=False)
    print(f"✓ Analyse overfitting: {output_prefix}_overfitting.csv")

    print(f"\n{'='*70}")
    print("TOUS LES RÉSULTATS ONT ÉTÉ SAUVEGARDÉS")
    print(f"{'='*70}")

    # Affichage du classement
    print(f"\n🏆 CLASSEMENT DES MODÈLES (par {grid_objects[list(grid_objects.keys())[0]].scoring}):")
    print("-" * 70)
    ranking = grid_df.sort_values('best_score', ascending=False)
    for i, row in ranking.iterrows():
        print(f"{i+1}. {row['model']:25s} - Score: {row['best_score']:.4f} - Temps: {row['time_seconds']:.1f}s")

    print("\n📊 DÉTECTION D'OVERFITTING:")
    print("-" * 70)
    for _, row in eval_df.iterrows():
        status = "⚠️  OVERFITTING" if row['overfitting'] == 'OUI' else "✓  Pas d'overfitting"
        print(f"{row['model']:25s}: {status} (écart: {row['accuracy_gap']:+.4f})")

# GridSearch avec grille réduite (rapide pour test)
# Utilisez param_grids='full' pour une recherche complète
grid_df, eval_df, grid_objects, detailed_reports = compare_models_gridsearch(
    X, y,
    param_grids='full',  # 'small', 'full', ou dict personnalisé
    test_size=0.15,
    cv=5,  # 3 pour test rapide, 5 recommandé
    scoring='f1_macro',
    random_state=82
)

# Sauvegarde
save_gridsearch_results(
    grid_df, eval_df, grid_objects, detailed_reports,
    output_prefix='gridsearch_results'
)

print("\n" + "="*70)
print("APERÇU DES MEILLEURS RÉSULTATS")
print("="*70)
print(grid_df.round(4).to_string(index=False))


GRIDSEARCH - COMPARAISON DES MODÈLES
Taille du dataset: 1470 échantillons
Train: 1249 | Test: 221
Distribution des classes: {False: np.int64(1233), True: np.int64(237)}

🔍 Grid Search pour DummyClassifier
Nombre de combinaisons à tester: 3
Métrique d'optimisation: f1_macro
Validation croisée: 5 folds
✓ Terminé en 1.69 secondes
📊 Meilleur score (f1_macro): 0.5202
🏆 Meilleurs paramètres:
   - strategy: stratified

📈 Évaluation sur le jeu de test:
   Train accuracy: 0.7462
   Test accuracy:  0.7149
   Écart accuracy: +0.0313
   Overfitting:    NON

🔍 Grid Search pour RandomForestClassifier
Nombre de combinaisons à tester: 432
Métrique d'optimisation: f1_macro
Validation croisée: 5 folds


KeyboardInterrupt: 